In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string

In [2]:
df_fake = pd.read_csv("https://raw.githubusercontent.com/KaiDMML/FakeNewsNet/master/dataset/gossipcop_fake.csv") 
df_true = pd.read_csv("https://raw.githubusercontent.com/KaiDMML/FakeNewsNet/master/dataset/gossipcop_real.csv")

In [3]:
df_fake.head(5)

,id,news_url,title,tweet_ids
0,gossipcop-2493749932,www.dailymail.co.uk/tvshowbiz/article-5874213/...,Did Miley Cyrus and Liam Hemsworth secretly ge...,284329075902926848\t284332744559968256\t284335...
1,gossipcop-4580247171,hollywoodlife.com/2018/05/05/paris-jackson-car...,Paris Jackson & Cara Delevingne Enjoy Night Ou...,992895508267130880\t992897935418503169\t992899...
2,gossipcop-941805037,variety.com/2017/biz/news/tax-march-donald-tru...,Celebrities Join Tax March in Protest of Donal...,853359353532829696\t853359576543920128\t853359...
3,gossipcop-2547891536,www.dailymail.co.uk/femail/article-3499192/Do-...,Cindy Crawford's daughter Kaia Gerber wears a ...,988821905196158981\t988824206556172288\t988825...
4,gossipcop-5476631226,variety.com/2018/film/news/list-2018-oscar-nom...,Full List of 2018 Oscar Nominations – Variety,955792793632432131\t955795063925301249\t955798...


In [4]:
df_true.head(5)

,id,news_url,title,tweet_ids
0,gossipcop-882573,https://www.brides.com/story/teen-mom-jenelle-...,Teen Mom Star Jenelle Evans' Wedding Dress Is ...,912371411146149888\t912371528343408641\t912372...
1,gossipcop-875924,https://www.dailymail.co.uk/tvshowbiz/article-...,Kylie Jenner refusing to discuss Tyga on Life ...,901989917546426369\t901989992074969089\t901990...
2,gossipcop-894416,https://en.wikipedia.org/wiki/Quinn_Perkins,Quinn Perkins,931263637246881792\t931265332022579201\t931265...
3,gossipcop-857248,https://www.refinery29.com/en-us/2018/03/19192...,I Tried Kim Kardashian's Butt Workout & Am For...,868114761723936769\t868122567910936576\t868128...
4,gossipcop-884684,https://www.cnn.com/2017/10/04/entertainment/c...,Celine Dion donates concert proceeds to Vegas ...,915528047004209152\t915529285171122176\t915530...


In [5]:
df_fake["label"] = "FAKE"
df_true["label"] = "TRUE"

In [6]:
df_fake.shape, df_true.shape

((5323, 5), (16817, 5))

In [7]:
df_fake_manual_testing = df_fake.tail(10)
for i in range(5322,5312,-1):
    df_fake.drop([i], axis = 0, inplace = True)
df_true_manual_testing = df_true.tail(10)
for i in range(16816,16806,-1):
    df_true.drop([i], axis = 0, inplace = True)

In [8]:
df_manual_testing = pd.concat([df_fake_manual_testing,df_true_manual_testing], axis = 0)
df_manual_testing.to_csv("manual_testing.csv")

In [9]:
k=pd.read_csv("manual_testing.csv")

In [10]:
k.head(10)

,Unnamed: 0,id,news_url,title,tweet_ids,label
0,5313,gossipcop-6844103138,www.foxnews.com/entertainment/2017/08/17/tom-c...,"Tom Cruise is 'diabolical,' says Leah Remini",897885664091492357\t897886019877523462\t897886...,FAKE
1,5314,gossipcop-7855996289,www.dailymail.co.uk/tvshowbiz/article-5765893/...,Angelina Jolie 'furious she can't take her chi...,999612553486577664,FAKE
2,5315,gossipcop-1761345281,www.cbs8.com/story/34968239/the-voice-blake-sh...,'The Voice': Blake Shelton Gets Jealous as Gwe...,845327633655255041\t845333603957518337\t845335...,FAKE
3,5316,gossipcop-7259603960,www.thebudgetbabe.com/categories/64-Jessica-Alba,Affordable Fashion & Style Blog,1237917051\t1432369293\t1515306142\t1939329774...,FAKE
4,5317,gossipcop-2194208436,www.hollywoodreporter.com/news/battle-sexes-pr...,'Battle of the Sexes' Premiere: Emma Stone on ...,909553415684067328\t909587019269013504\t909623...,FAKE
5,5318,gossipcop-6702260693,www.huffingtonpost.com/2012/09/11/september-11...,September 11: Celebrities Remember 9/11 (TWEETS),245643768638894080,FAKE
6,5319,gossipcop-6051845337,www.dailymail.co.uk/news/article-4915674/NASCA...,NASCAR owners threaten to fire drivers who pro...,912048333413330944\t912048571482087424\t912049...,FAKE
7,5320,gossipcop-2435526162,www.telegraph.co.uk/men/the-filter/7-signs-dav...,The 7 signs that David Beckham is definitely h...,897794716447539200\t897804460830928896\t897842...,FAKE
8,5321,gossipcop-4576152851,www.vanityfair.com/style/2016/09/ryan-gosling-...,Ryan Gosling and Eva Mendes Did Not Get Marrie...,778678901572710400\t778681718714740736\t778683...,FAKE
9,5322,gossipcop-919334865,www.lifeandstylemag.com/posts/jamie-foxx-katie...,Jamie Foxx Spends the Day With Katie Holmes an...,913137595424608258\t913139996059717632\t913146...,FAKE


In [11]:
df_marge = pd.concat([df_fake, df_true], axis =0 )
df_marge.head(10)

,id,news_url,title,tweet_ids,label
0,gossipcop-2493749932,www.dailymail.co.uk/tvshowbiz/article-5874213/...,Did Miley Cyrus and Liam Hemsworth secretly ge...,284329075902926848\t284332744559968256\t284335...,FAKE
1,gossipcop-4580247171,hollywoodlife.com/2018/05/05/paris-jackson-car...,Paris Jackson & Cara Delevingne Enjoy Night Ou...,992895508267130880\t992897935418503169\t992899...,FAKE
2,gossipcop-941805037,variety.com/2017/biz/news/tax-march-donald-tru...,Celebrities Join Tax March in Protest of Donal...,853359353532829696\t853359576543920128\t853359...,FAKE
3,gossipcop-2547891536,www.dailymail.co.uk/femail/article-3499192/Do-...,Cindy Crawford's daughter Kaia Gerber wears a ...,988821905196158981\t988824206556172288\t988825...,FAKE
4,gossipcop-5476631226,variety.com/2018/film/news/list-2018-oscar-nom...,Full List of 2018 Oscar Nominations – Variety,955792793632432131\t955795063925301249\t955798...,FAKE
5,gossipcop-5189580095,www.townandcountrymag.com/society/tradition/a1...,Here's What Really Happened When JFK Jr. Met P...,890253005299351552\t890401381814870016\t890491...,FAKE
6,gossipcop-9588339534,www.foxnews.com/entertainment/2016/12/16/bigge...,Biggest celebrity scandals of 2016,683226380742557696\t748604615503929345\t748604...,FAKE
7,gossipcop-8753274298,www.eonline.com/news/958257/caitlyn-jenner-add...,Caitlyn Jenner Addresses Rumored Romance With ...,1026891446081728512\t1026891745219543043\t1026...,FAKE
8,gossipcop-8105333868,www.inquisitr.com/3871816/taylor-swift-reporte...,Taylor Swift Reportedly Reacts To Tom Hiddlest...,818928533569437697\t819100640878202880\t819174...,FAKE
9,gossipcop-2803748870,www.huffingtonpost.com/entry/kate-mckinnon-the...,"For The Love Of God, Why Can't Anyone Write Ka...",816030248190046212\t816030859484626947\t816049...,FAKE


In [12]:
df_marge.columns

Index(['id', 'news_url', 'title', 'tweet_ids', 'label'], dtype='object')

In [13]:
df = df_marge.drop(["id", "tweet_ids","news_url"], axis = 1)
df.head()

,title,label
0,Did Miley Cyrus and Liam Hemsworth secretly ge...,FAKE
1,Paris Jackson & Cara Delevingne Enjoy Night Ou...,FAKE
2,Celebrities Join Tax March in Protest of Donal...,FAKE
3,Cindy Crawford's daughter Kaia Gerber wears a ...,FAKE
4,Full List of 2018 Oscar Nominations – Variety,FAKE


In [14]:
df.isnull().sum()

title    0
label    0
dtype: int64

In [15]:
#SHUFFELING


In [16]:
df = df.sample(frac = 1)
df.head()

,title,label
8603,Gigi Hadid's Makeup Artist Makes Pastel Eye Sh...,TRUE
230,Nicole Kidman and Keith Urban Gush Over Daughters,FAKE
2166,Heidi Montag And Spencer Pratt Talk Future Pla...,TRUE
13837,The Good Fight is coming back in March,TRUE
16178,'Outlander': Inside the Making of the Iconic P...,TRUE


In [17]:
df.reset_index(inplace = True)
df.drop(["index"], axis = 1, inplace = True)

In [18]:
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) 
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)    
    return text

In [19]:
df["title"] = df["title"].apply(wordopt)

In [20]:
x = df["title"]
y = df["label"]

In [21]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state = 0)

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [23]:
vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

In [24]:
from sklearn.linear_model import LogisticRegression

In [25]:
LR=LogisticRegression()
LR.fit(xv_train,y_train) 

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [26]:
pred_lr=LR.predict(xv_test)

In [27]:
LR.score(xv_test, y_test)

0.8437613019891501

In [28]:
print(classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

        FAKE       0.79      0.47      0.59      1318
        TRUE       0.85      0.96      0.90      4212

    accuracy                           0.84      5530
   macro avg       0.82      0.72      0.75      5530
weighted avg       0.84      0.84      0.83      5530



In [29]:
#Decision tree classification

In [30]:
from sklearn.tree import DecisionTreeClassifier

In [31]:
DT = DecisionTreeClassifier()
DT.fit(xv_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [32]:
pred_dt = DT.predict(xv_test)

In [33]:
DT.score(xv_test, y_test)

0.7853526220614828

In [34]:
print(classification_report(y_test, pred_dt))

              precision    recall  f1-score   support

        FAKE       0.55      0.54      0.55      1318
        TRUE       0.86      0.86      0.86      4212

    accuracy                           0.79      5530
   macro avg       0.70      0.70      0.70      5530
weighted avg       0.78      0.79      0.78      5530



In [35]:
### Gradient Boosting Classifier

In [36]:
from sklearn.ensemble import GradientBoostingClassifier

In [37]:
GBC = GradientBoostingClassifier(random_state=0)
GBC.fit(xv_train, y_train)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=0, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [38]:
pred_gbc = GBC.predict(xv_test)

In [39]:
GBC.score(xv_test, y_test)

0.8188065099457504

In [40]:
print(classification_report(y_test, pred_gbc))

              precision    recall  f1-score   support

        FAKE       0.80      0.32      0.46      1318
        TRUE       0.82      0.98      0.89      4212

    accuracy                           0.82      5530
   macro avg       0.81      0.65      0.67      5530
weighted avg       0.82      0.82      0.79      5530



In [41]:
### Random Forest Classifier

In [42]:
from sklearn.ensemble import RandomForestClassifier

In [43]:
RFC = RandomForestClassifier(random_state=0)
RFC.fit(xv_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [44]:
pred_rfc = RFC.predict(xv_test)

In [45]:
RFC.score(xv_test, y_test)

0.8361663652802893

In [46]:
print(classification_report(y_test, pred_rfc))

              precision    recall  f1-score   support

        FAKE       0.74      0.48      0.58      1318
        TRUE       0.85      0.95      0.90      4212

    accuracy                           0.84      5530
   macro avg       0.80      0.71      0.74      5530
weighted avg       0.83      0.84      0.82      5530



In [47]:
def output_lable(n):
    if n == 0:
        return "Fake News"
    elif n == 1:
        return "Not A Fake News"
    
def manual_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(wordopt) 
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)
    pred_LR = LR.predict(new_xv_test)
    pred_DT = DT.predict(new_xv_test)
    pred_GBC = GBC.predict(new_xv_test)
    pred_RFC = RFC.predict(new_xv_test)

    return print("\n\nLR Prediction: {} \nDT Prediction: {} \nGBC Prediction: {} \nRFC Prediction: {}".format(output_lable(pred_LR[0]), 
                                                                                                              output_lable(pred_DT[0]), 
                                                                                                              output_lable(pred_GBC[0]), 
                                                                                                              output_lable(pred_RFC[0])))

In [48]:
news = str(input())
manual_testing(news)

After last week's shock announcement that Kylie Jenner wasn't just very much pregnant, she had actually birthed a real human child, the world went wild speculating on what the baby's name was going to be.  We know now that Kylie's daughter is called Stormi Webster, but until that record-breaking announcement came, the most convincing theory was that the new baby was going to be called something butterfly-related. Eagle-eyed fans spotted all sorts of butterfly references in the 11-minute long announcement video, and it all seemed very plausible.


LR Prediction: Not A Fake News 
DT Prediction: Not A Fake News 
GBC Prediction: Not A Fake News 
RFC Prediction: Not A Fake News
